In [2]:
##### main script to start the application
#required libraries for performing data analysis and predictions on the imdb data
from IPython.core.display import clear_output
from time import sleep
import ast 
from random import randint
from time import time 
import csv,sqlite3
from requests import get
from bs4 import BeautifulSoup
import calendar
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import seaborn as sns
import tkinter
from tkinter import messagebox, Label, Button,Tk,Entry,Radiobutton
import pickle,sys
from warnings import warn
import matplotlib.pyplot as plot
import seaborn as sns
%matplotlib qt

#function to to retrieve the imdb list based on the give year

def imdbListByYear(startYear,endYear):
        headers = {"Accept-Language": "en-US, en;q=0.5"}
        years = [str(i) for i in range(startYear,endYear)]
        pages = [str(i) for i in range(1,4)]
        
        # declaring the lists to store data in
        imdbtitle = []
        imdbyear = []
        imdbrating = []
        imdbmscore = []
        imdbvote = []
        
        
       # Preparing the monitoring of the loop
        start_time = time()
        requests = 0

        # For every year in the interval startYear -endYear
        for year in years:

            # For every page in the interval 1-3
            for page in pages:

                # Make a get request
                response = get('https://www.imdb.com/search/title?release_date=' + year)

                # Pause the loop
                sleep(randint(8,15))

                # Monitor the requests
                requests += 1
                elapsed_time = time() - start_time
                print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
                clear_output(wait = True)

                # Throw a warning for non-200 status codes
                if response.status_code != 200:
                    warn('Request: {}; Status code: {}'.format(requests, response.status_code))

                # Break the loop if the number of requests is greater than expected
                if requests > 72:
                    warn('Number of requests was greater than expected.')  
                    break 

                # Parse the content of the request with BeautifulSoup
                page_html = BeautifulSoup(response.text, 'html.parser')

                # Select all the 50 movie containers from a single page
                movies = page_html.find_all('div', class_ = 'lister-item mode-advanced')

                # For every movie of these 50
                for movie in movies:
                    # If the movie has a Metascore, then:
                    if movie.find('div', class_ = 'ratings-metascore') is not None:

                        # Scrape the name
                        name = movie.h3.a.text
                        imdbtitle.append(name)

                        # Scrape the year 
                        syear = movie.h3.find('span', class_ = 'lister-item-year').text
                        imdbyear.append(year)

                        # Scrape the IMDB rating
                        imdb = float(movie.strong.text)
                        imdbrating.append(imdb)

                        # Scrape the Metascore
                        m_score = movie.find('span', class_ = 'metascore').text
                        imdbmscore.append(int(m_score))

                        # Scrape the number of votes
                        vote = movie.find('span', attrs = {'name':'nv'})['data-value']
                        imdbvote.append(int(vote))
        imdbListYear = pd.DataFrame()
        imdbListYear = pd.DataFrame({'movie': imdbtitle,
                                          'year': imdbyear,
                                          'rating': imdbrating,
                                          'metascore':imdbmscore,
                                          'votes':imdbvote})
        imdbmovielist = imdbListYear[['movie', 'year', 'rating', 'metascore', 'votes']]
        #imdbmovielist.head()
        return (imdbmovielist)

#function to to retrieve the imdb list based on the give month  
def imdbListByMonth(rstartDate,rendDate):
            
        #declaring the lists to store data in
        imdbtitle = []
        imdbmonth  = []
        imdbrating = []
        imdbmscore = []
        imdbvote = []
        
        pages = [str(i) for i in range(1,3)]
        # Preparing the monitoring of the loop
        start_time = time()
        requests = 0

        # For every year in the interval startYear -endYear
        for month in monthRange(rstartDate, rendDate):

            # For every page in the interval 1-2
            for page in pages:

                month = str(month)
                print(month)
                clear_output(wait = True)
                # Make a get request
                response = get('http://www.imdb.com/search/title?release_date=' + month)

                # Pause the loop
                sleep(randint(8,15))

                # Monitor the requests
                requests += 1
                elapsed_time = time() - start_time
                print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
                clear_output(wait = True)

                # Throw a warning for non-200 status codes
                if response.status_code != 200:
                    warn('Request: {}; Status code: {}'.format(requests, response.status_code))

                # Break the loop if the number of requests is greater than expected
                if requests > 220:
                    warn('Number of requests was greater than expected.')  
                    break 

                # Parse the content of the request with BeautifulSoup
                page_html = BeautifulSoup(response.text, 'html.parser')

                # Select all the 50 movie containers from a single page
                movies = page_html.find_all('div', class_ = 'lister-item mode-advanced')

                # For every movie of these 50
                for movie in movies:
                    # If the movie has a Metascore, then:
                    if movie.find('div', class_ = 'ratings-metascore') is not None:

                        # Scrape the name
                        name = movie.h3.a.text
                        imdbtitle.append(name)

                        # Scrape the month
                        smonth = movie.h3.find('span', class_ = 'lister-item-year').text
                        imdbmonth.append(month)

                        # Scrape the IMDB rating
                        imdb = float(movie.strong.text)
                        imdbrating.append(imdb)

                        # Scrape the Metascore
                        m_score = movie.find('span', class_ = 'metascore').text
                        imdbmscore.append(int(m_score))

                        # Scrape the number of votes
                        vote = movie.find('span', attrs = {'name':'nv'})['data-value']
                        imdbvote.append(int(vote))
        
        imdbListMonth = pd.DataFrame()
        imdbListMonth = pd.DataFrame({'movie': imdbtitle,
                                          'date': imdbmonth,
                                          'rating': imdbrating,
                                          'metascore':imdbmscore,
                                          'votes':imdbvote})
        imdbmovielist = imdbListMonth[['movie', 'date', 'rating', 'metascore', 'votes']]
        #imdbmovielist.head()
        return (imdbmovielist)

#function to get the date based on the input entered by the user   
def daysInMonth(date):
    return calendar.monthrange(date.year, date.month)[1]

def monthRange(startDate, endDate):
    forward = endDate >= startDate
    finish = False
    date = startDate

    while not finish:
        yield date.date()
        if forward:
            days = daysInMonth(date)
            date = date + timedelta(days=days)            
            finish = date > endDate
        else:
            tempDate = date.replace(day=1) - timedelta(days=1)
            date = (tempDate.replace(day=date.day))
            finish = date < endDate

#function to do classification on the monthly data retrieved from the IMDB website          
def dataAnalysisMonthly(imdbmovielist):
    
    ## retrieving the votes & metascore and ratings and storing X and y respectively
    from sklearn.preprocessing import MinMaxScaler
    X = imdbmovielist.iloc[:,3:5]
    y = imdbmovielist['rating']
    
    #pre-processing the data
    scaler = MinMaxScaler().fit(X)
    X= scaler.transform(X)
    y = np.asarray(imdbmovielist['rating'], dtype="|S6")

    
    ## importing train test split
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
    
    ## importing Multinomial Naive Bayes and fitting the classifier
    from sklearn.naive_bayes import MultinomialNB
    nb = MultinomialNB()
    nb.fit(X_train, y_train)
    ## predicting the rating against the test data 
    predictions = nb.predict(X_test)
    ## importing the accuracy score and determining the accuracy by testing the predicted rating with test data original rating
    from sklearn.metrics import accuracy_score
    mnbAcc = accuracy_score(y_test,predictions)
    
    ## importing Gaussian Naive Bayes and fitting the classifier
    from sklearn.naive_bayes import GaussianNB
    nb = GaussianNB()
    nb.fit(X_train, y_train)
    ## predicting the rating against the test data 
    predictions = nb.predict(X_test)
    ## importing the accuracy score and determining the accuracy by testing the predicted rating with test data original rating
    from sklearn.metrics import accuracy_score
    gnbAcc = accuracy_score(y_test,predictions)
    
    ## importing KNN and fitting the classifier
    from sklearn.neighbors import KNeighborsClassifier
    ## building a classifier using KNN 
    classifier = KNeighborsClassifier(algorithm='brute')
    ##fitting the classifier
    classifier.fit(X_train, y_train)
    ## predicting the rating against the test data
    predictions = classifier.predict(X_test)
    ## importing the accuracy score and determining the accuracy by testing the predicted rating with test data original rating
    from sklearn.metrics import accuracy_score
    knnAcc = accuracy_score(y_test,predictions)
    
    ## importing Decision tree and fitting the classifier
    from sklearn.tree import DecisionTreeClassifier
    ##building a decision tree classifier
    classifier=DecisionTreeClassifier(random_state=0)
    ## fitting the classifier
    classifier.fit(X_train, y_train)
    ## predicting the rating against the test data
    predictions = classifier.predict(X_test)
    ## importing the accuracy score and determining the accuracy by testing the predicted rating with test data original rating
    from sklearn.metrics import accuracy_score
    dtAcc = accuracy_score(y_test,predictions)
    
    ## importing the Bernoulli Naive Bayes
    from sklearn.naive_bayes import BernoulliNB
    ## Building the classifier
    classifier = BernoulliNB()
    ## fitting the Bernoulli NB classifier
    classifier.fit(X_train, y_train)
    ## predicting the rating against the test data 
    predictions = classifier.predict(X_test)
    ## importing the accuracy score and determining the accuracy by testing the predicted rating with test data original rating
    from sklearn.metrics import accuracy_score
    bnbAcc = accuracy_score(y_test,predictions)
    
    ## importing the extra tree classifier
    from sklearn.tree import ExtraTreeClassifier
    ## Building the classifier
    classifier = ExtraTreeClassifier()
    ## fitting the extra tree classifier
    classifier.fit(X_train, y_train)
    ## predicting the rating against the test data 
    predictions = classifier.predict(X_test)
    ## importing the accuracy score and determining the accuracy by testing the predicted rating with test data original rating
    from sklearn.metrics import accuracy_score
    rfAcc = accuracy_score(y_test,predictions)
        
    #print (mnbAcc,gnbAcc,knnAcc,dtAcc,bnbAcc,rfAcc)
    bestAccuracy = max(mnbAcc,gnbAcc,knnAcc,dtAcc,bnbAcc,rfAcc)
    
    return bestAccuracy,predictions,X_test,mnbAcc,gnbAcc,knnAcc,dtAcc,bnbAcc,rfAcc

#function to do classification on the yearly data retrieved from the IMDB website  
def dataAnalysisYearly(imdbmovielist):
    
    ## retrieving the votes & metascore and ratings and storing X and y respectively
    from sklearn.preprocessing import MinMaxScaler
    #print(imdbmovielist)
    X = imdbmovielist.iloc[:,3:5]
    y = imdbmovielist['rating']

    #pre-processing the data
    scaler = MinMaxScaler().fit(X)
    X= scaler.transform(X)
    y = np.asarray(imdbmovielist['rating'], dtype="|S6")

    
    ## importing train test split
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
    
    ## importing Multinomial Naive Bayes and fitting the classifier
    from sklearn.naive_bayes import MultinomialNB
    nb = MultinomialNB()
    nb.fit(X_train, y_train)
    ## predicting the rating against the test data 
    predictions = nb.predict(X_test)
    ## importing the accuracy score and determining the accuracy by testing the predicted rating with test data original rating
    from sklearn.metrics import accuracy_score
    mnbAcc = accuracy_score(y_test,predictions)
    
    ## importing Gaussian Naive Bayes and fitting the classifier
    from sklearn.naive_bayes import GaussianNB
    nb = GaussianNB()
    nb.fit(X_train, y_train)
    ## predicting the rating against the test data 
    predictions = nb.predict(X_test)
    ## importing the accuracy score and determining the accuracy by testing the predicted rating with test data original rating
    from sklearn.metrics import accuracy_score
    gnbAcc = accuracy_score(y_test,predictions)
    
    ## importing KNN and fitting the classifier
    from sklearn.neighbors import KNeighborsClassifier
    ## building a classifier using KNN 
    classifier = KNeighborsClassifier(algorithm='brute')
    ##fitting the classifier
    classifier.fit(X_train, y_train)
    ## predicting the rating against the test data
    predictions = classifier.predict(X_test)
    ## importing the accuracy score and determining the accuracy by testing the predicted rating with test data original rating
    from sklearn.metrics import accuracy_score
    knnAcc = accuracy_score(y_test,predictions)
    
    ## importing Decision tree and fitting the classifier
    from sklearn.tree import DecisionTreeClassifier
    ##building a decision tree classifier
    classifier=DecisionTreeClassifier(random_state=0)
    ## fitting the classifier
    classifier.fit(X_train, y_train)
    ## predicting the rating against the test data
    predictions = classifier.predict(X_test)
    ## importing the accuracy score and determining the accuracy by testing the predicted rating with test data original rating
    from sklearn.metrics import accuracy_score
    dtAcc = accuracy_score(y_test,predictions)
    
    ## importing the Bernoulli Naive Bayes
    from sklearn.naive_bayes import BernoulliNB
    ## Building the classifier
    classifier = BernoulliNB()
    ## fitting the Bernoulli NB classifier
    classifier.fit(X_train, y_train)
    ## predicting the rating against the test data 
    predictions = classifier.predict(X_test)
    ## importing the accuracy score and determining the accuracy by testing the predicted rating with test data original rating
    from sklearn.metrics import accuracy_score
    bnbAcc = accuracy_score(y_test,predictions)
    
    ## importing the extra tree classifier
    from sklearn.tree import ExtraTreeClassifier
    ## Building the classifier
    classifier = ExtraTreeClassifier()
    ## fitting the extra tree classifier
    classifier.fit(X_train, y_train)
    ## predicting the rating against the test data 
    predictions = classifier.predict(X_test)
    ## importing the accuracy score and determining the accuracy by testing the predicted rating with test data original rating
    from sklearn.metrics import accuracy_score
    rfAcc = accuracy_score(y_test,predictions)
    
    #print (mnbAcc,gnbAcc,knnAcc,dtAcc,bnbAcc,rfAcc)
    bestAccuracy = max(mnbAcc,gnbAcc,knnAcc,dtAcc,bnbAcc,rfAcc)
    
    return bestAccuracy,predictions,X_test,mnbAcc,gnbAcc,knnAcc,dtAcc,bnbAcc,rfAcc

#function for Gui which builds the best classifier using the yearly data retrieved from the imdb website
def classificationYearly(imdbListYear) :
    
    #declaring the titles and geometry for the gui frame
    root = Tk()
    var = tkinter.StringVar()
    root.title("Perform Data Analysis and Build Classifier")
    root.geometry("1350x1250")
    
    #function to plot the scatter plot
    def scatterPlotYear(imdbmovielist):
        #scatterPlot.configure(state = "disabled")
        import matplotlib.pyplot as plot
        import seaborn as sns
        %matplotlib qt
        
        #scatter plot for votes and rating
        imdbmovielist.plot.scatter(x='votes',y='rating',c='DarkBlue',figsize=(10, 10),title ="Rating vs Votes")
        
        #scatter plot for metascore and rating
        imdbmovielist.plot.scatter(x='metascore',y='rating',c='DarkBlue',figsize=(10, 10),title ="Rating vs Metascore")
        
        #pairplot for votes,metascore and rating
        sns.pairplot(vars=["metascore","votes"], data=imdbmovielist, hue="rating", size=5)
        
        #histogram for votes,metascore and rating
        fig, axes = plot.subplots(nrows = 1, ncols = 3, figsize = (16,6))
        ax1, ax2,ax3 = fig.axes

        ax1.hist(imdbmovielist['rating'], bins = 10) # bin range = 10
        ax1.set_title('IMDB Rating')

        ax2.hist(imdbmovielist['metascore'], bins = 10) # bin range = 10
        ax2.set_title('Metascore')

        ax3.hist(imdbmovielist['votes'], bins = 10) # bin range = 10
        ax3.set_title('Votes')

        for ax in fig.axes:
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
        plot.show()
        
    #function to build accuracy chart based on the accuracy of each classifier built
    def accuracyChart(mnbAcc,gnbAcc,knnAcc,dtAcc,bnbAcc,rfAcc):
        def accuracy(mnbAcc,gnbAcc,knnAcc,dtAcc,bnbAcc,rfAcc):
            import matplotlib.pyplot as plot
            %matplotlib qt
            objects = ('MultinomialNB','GaussianNB','KNN','Descicion Tree','BernoulliNB','Extra Tree')
            y_pos = np.arange(len(objects))
            performance = [mnbAcc,gnbAcc,knnAcc,dtAcc,bnbAcc,rfAcc]

            plot.figure(figsize=(13,10))
            plot.bar(y_pos, performance, align='center', alpha=0.5,color = 'darkgreen')
            plot.xticks(y_pos, objects)
            plot.xlabel('Types of classifiers')
            plot.ylabel('Accuracy on the model fit')
            plot.title('IMDB Rating Prediction- Accuracy of the model fit')

            plot.show()

        accuracyChart = Button(text="\nView the best classifier fit", command= lambda arg1 = mnbAcc,
                               arg2 = gnbAcc,arg3 = knnAcc,arg4 = dtAcc,arg5 = bnbAcc,arg6 = rfAcc: accuracy(arg1,arg2,arg3,arg4,arg5,arg6))   
        accuracyChart.pack()

    #function which prints first 10 rows of the predicted IMDB ratings
    def classifier(imdbListYear) :
        
        classification.configure(state="disabled")
        bestAccuracy,predictions,X_test,mnbAcc,gnbAcc,knnAcc,dtAcc,bnbAcc,rfAcc = dataAnalysisYearly(imdbListYear)
        
        label6 = Label(root, text="\nThe confidence of the best classifier fit is "+ str(round((bestAccuracy*100),2))+"%")
        label6.pack()
        label1 = Label(root, text="\n********** PREDICTED IMDB RATING FROM THE CLASSIFIER FITTED ********** ")
        label1.pack()
        label2 = Label(root, text="------------------------------------------------------------------------ ")
        label2.pack()
        label3 = Label(root, text="\tMetascore\t Votes\t\tIMDB Rating ")
        label3.pack()
        label4 = Label(root, text="------------------------------------------------------------------------ ")
        label4.pack()
        for i in range(len(X_test[0:10])):
            s = str(predictions[i])
            x = str(X_test[i])
            startx = "["
            endx   ="]"
            start = "b'"
            end = "'"
            my_string = s[s.find(start)+len(start):s.rfind(end)]
            x_string = x[x.find(startx)+len(startx):x.rfind(endx)]
            metascore= x_string[:10]
            votes= x_string[10:]
            label5 = Label(root, text="%s\t%s\t%s" % (metascore,votes, my_string))
            label5.pack()
        accuracyChart(mnbAcc,gnbAcc,knnAcc,dtAcc,bnbAcc,rfAcc)
        label7 = Label(root, text="\n\nnote : the data of metascore and votes are pre-processed and transformed")
        label7.pack()
                      
    classifierLabel = Label(root, text="Build the best classifier and predict the IMDB Rating based on metascore and votes ")
    classification = Button(text="Start the Classification Process", command= lambda arg1=imdbListYear :classifier(arg1))

    classifierLabel.pack()
    classification.pack()
    scatterPlot = Button(text="Scatter Plot - Metascore,Votes,Rating", command= lambda arg1 = imdbListYear: scatterPlotYear(arg1))   
    scatterPlot.pack()
    root.mainloop()
 
 #function for Gui which builds the best classifier using the monthly data retrieved from the imdb website
def classificationMonthly(imdbListMonth) :
    
    #declaring the titles and geometry for the gui frame
    root = Tk()
    var = tkinter.StringVar()
    root.title("Perform Data Analysis and Build Classifier")
    root.geometry("1350x1050")
    
     #function to plot the scatter plot
    def scatterPlotMonth(imdbmovielist):
        #scatterPlot.configure(state = "disabled")
        import matplotlib.pyplot as plot
        import seaborn as sns
        %matplotlib qt
        
         #scatter plot for votes and rating
        imdbmovielist.plot.scatter(x='votes',y='rating',c='DarkBlue',figsize=(10, 10),title ="Rating vs Votes")
        
         #scatter plot for metascore and rating
        imdbmovielist.plot.scatter(x='metascore',y='rating',c='DarkBlue',figsize=(10, 10),title ="Rating vs Metascore")
        
        #pairplot for votes,metascore and rating
        sns.pairplot(vars=["metascore","votes"], data=imdbmovielist, hue="rating", size=5)
        
        #histogram for votes,metascore and rating
        fig, axes = plot.subplots(nrows = 1, ncols = 3, figsize = (16,6))
        ax1, ax2,ax3 = fig.axes

        ax1.hist(imdbmovielist['rating'], bins = 10) # bin range = 10
        ax1.set_title('IMDB Rating')

        ax2.hist(imdbmovielist['metascore'], bins = 10) # bin range = 10
        ax2.set_title('Metascore')

        ax3.hist(imdbmovielist['votes'], bins = 10) # bin range = 10
        ax3.set_title('Votes')

        for ax in fig.axes:
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
        plot.show()
        
        
    #function to build accuracy chart based on the accuracy of each classifier built
    def accuracyChart(mnbAcc,gnbAcc,dtAcc,bnbAcc,rfAcc):
        def accuracy(mnbAcc,gnbAcc,dtAcc,bnbAcc,rfAcc):
            import matplotlib.pyplot as plot
            %matplotlib qt
            objects = ('MultinomialNB','GaussianNB','Descicion Tree','BernoulliNB','Extra Tree')
            y_pos = np.arange(len(objects))
            performance = [mnbAcc,gnbAcc,dtAcc,bnbAcc,rfAcc]

            plot.figure(figsize=(13,10))
            plot.bar(y_pos, performance, align='center', alpha=0.5,color = 'darkgreen')
            plot.xticks(y_pos, objects)
            plot.ylabel('Accuracy on the model fit')
            plot.title('IMDB Rating Prediction- Accuracy of the model fit')

            plot.show()

        accuracyChart = Button(text="\nView the best classifier fit", command= lambda arg1 = mnbAcc,
                               arg2 = gnbAcc,arg3 = dtAcc,arg4 = bnbAcc,arg5 = rfAcc: accuracy(arg1,arg2,arg3,arg4,arg5))   
        accuracyChart.pack()
        
    #function which prints first 10 rows of the predicted IMDB ratings
    def classifier(imdbListMonth) :
        
        classification.configure(state="disabled")    
        bestAccuracy,predictions,X_test,mnbAcc,gnbAcc,knnAcc,dtAcc,bnbAcc,rfAcc = dataAnalysisMonthly(imdbListMonth)
        
        label6 = Label(root, text="\n\nThe confidence of the best classifier fit is "+ str(round((bestAccuracy*100),2))+"%")
        label6.pack()
        label1 = Label(root, text="\n********** PREDICTED IMDB RATING FROM THE CLASSIFIER FITTED ********** ")
        label1.pack()
        label2 = Label(root, text="------------------------------------------------------------------------ ")
        label2.pack()
        label3 = Label(root, text="\tMetascore\t Votes\t\tIMDB Rating ")
        label3.pack()
        label4 = Label(root, text="------------------------------------------------------------------------ ")
        label4.pack()
        for i in range(12,18):
            s = str(predictions[i])
            x = str(X_test[i])
            startx = "["
            endx   ="]"
            start = "b'"
            end = "'"
            my_string = s[s.find(start)+len(start):s.rfind(end)]
            x_string = x[x.find(startx)+len(startx):x.rfind(endx)]
            metascore = x_string[:10]
            votes = x_string[10:]
            label5 = Label(root, text="%s\t%s\t%s" % (metascore,votes, my_string))
            label5.pack()
        accuracyChart(mnbAcc,gnbAcc,dtAcc,bnbAcc,rfAcc)
        label7 = Label(root, text="\n\nnote : the data of metascore and votes are pre-processed and transformed")
        label7.pack()     
               
    classifierLabel = Label(root, text="Build the best classifier and predict the IMDB Rating based on metascore and votes ")
    classification = Button(text="Start the Classification Process :", command= lambda arg1=imdbListMonth :classifier(arg1))

    classifierLabel.pack()
    classification.pack()

    scatterPlot = Button(text="Scatter Plot - Metascore,Votes,Rating", command= lambda arg1 = imdbListMonth: scatterPlotMonth(arg1))   
    scatterPlot.pack()

#function to retrieve the IMDB data based on the inputs given by the user

def viewIMDBData():
    
    #declaring the titles and geometry for the gui frame
    root = Tk()
    var = tkinter.StringVar()
    root.title("Retrieving the IMDB Data")
    root.geometry("1350x1050")
    
    #function to check whether the user input years are valid
    def valid_year(year):
        if year and year.isdigit():
            if int(year) >=1900 and int(year) <=2020:
                return True
            
    #function to check whether the user input months are valid
    def valid_month(month):
        if month and month.isdigit():
            if int(month) >=1 and int(month) <=12:
                return True
           
    #function to check whether the user input dates are valid
    def valid_date(date):
        if date and date.isdigit():
            if int(date) >=1 and int(date) <=31:
                return True
            
    
    #functions that retrieves the imdb data based on the start year and end yera entered by the user
    def retrieveIMDBYear(startYear,endYear) :
       
        if (startYear.get() !='' and endYear.get() !='' and valid_year(startYear.get()) and valid_year(endYear.get()) ):
            messagebox.showinfo("Please Wait...","Please wait while the Data is being retrieved from the IMDB website !!")
            startYear = startYear.get()
            endYear   = endYear.get()
            startYear = int(startYear)
            endYear   = int(endYear)

            imdbListYear=imdbListByYear(startYear,endYear)
            
            messagebox.showinfo("SUCCESS","The IMDB list between the years "+ str(startYear)+" and " 
                                  + str(endYear) +" has been successfully retrieved !!")
            root.withdraw()
            root.destroy()
            imdbListYear.to_csv('imdbListYear.csv')
            imdbListYear = pd.read_csv('imdbListYear.csv')
            classificationYearly(imdbListYear)

        else:
            messagebox.showerror("INVALID","Please enter valid details!")
            
    #functions that retrieves the imdb data based on the start month and end month entered by the user
    def retrieveIMDBMonth(startYear,startMonth,startDate,endYear,endMonth,endDate) :
            
        if (startYear.get() !='' and startMonth.get() !='' and startDate.get() !='' and endYear.get() != '' and  endMonth != ''
            and endDate != '' and valid_year(startYear.get()) and valid_year(endYear.get()) and valid_month(startMonth.get())
            and valid_month(endMonth.get()) and valid_date(startDate.get()) and valid_date(endDate.get())):
            messagebox.showinfo("Please Wait...","Please wait while the Data is being retrieved from the IMDB website !!")
            startYear = int(startYear.get())
            startMonth= int(startMonth.get())
            startDate = int(startDate.get())
            endYear   = int(endYear.get())
            endMonth  = int(endMonth.get())
            endDate   = int(endDate.get())
            
            rstartDate = datetime(startYear,startMonth,startDate)
            rendDate = datetime(endYear,endMonth,endDate)

            imdbListMonth=imdbListByMonth(rstartDate,rendDate)
            
            messagebox.showinfo("SUCCESS","The IMDB list for the months between "+ str(startYear)+"/"+str(startMonth)+"/"+
                                str(startDate)+" and " 
                                  + str(endYear) +"/"+str(endMonth)+"/"+
                                str(endDate) +" has been successfully retrieved !!")
            root.withdraw()
            root.destroy()
            imdbListMonth.to_csv('imdbListMonth.csv')
            imdbListMonth = pd.read_csv('imdbListMonth.csv')
            classificationMonthly(imdbListMonth)

        else:
            messagebox.showerror("INVALID","Please enter valid details!")
            
    #function for the radiobutton selected
    def selected():

        if (var.get() == "1" ):
            monthlyList.configure(state= "disabled")
            startYearLabel = Label(root, text="\nPlease enter the start year from which you want to view the IMDB titles list: ")
            startYear = Entry(root)
            endYearLabel = Label(root, text="Please enter the end year until which you want to view the IMDB titles list: ")
            endYear = Entry(root)
            retrieveData = Button(text="Retrieve Data", command=lambda arg1=startYear, arg2=endYear : retrieveIMDBYear(arg1,arg2))
            #= lambda arg="live", kw="as the" : a_new_method(arg, opt1=kw)
            startYearLabel.pack()
            startYear.pack()
            endYearLabel.pack()
            endYear.pack()
            retrieveData.pack()
                         
        elif (var.get() == "2" ):
            yearlyList.configure(state= "disabled")
            yearLabel = Label(root, text="Please enter the yyyy/mm/dd for the start date from which you want to view the IMDB movie list: ")
            startYearLabel = Label(root, text="\nEnter start year :")
            startYear = Entry(root)  
            startMonthLabel = Label(root, text="Enter start month :") 
            startMonth = Entry(root)
            startDateLabel = Label(root, text="Enter start date :")
            startDate = Entry(root)
            
            
            endYearLabel = Label(root, text="Enter end year :")
            endYear = Entry(root) 
            endMonthLabel = Label(root, text="Enter end month :")
            endMonth = Entry(root)
            endDateLabel = Label(root, text="Enter end date :")
            endDate = Entry(root)
            
            
            retrieveData = Button(text="Retrieve Data", command=lambda arg1=startYear, arg2=startMonth ,arg3 = startDate, arg4 = endYear,
                                  arg5 = endMonth ,arg6 = endDate : retrieveIMDBMonth(arg1,arg2,arg3,arg4,arg5,arg6))
            
            yearLabel.pack()
            startYearLabel.pack()
            startYear.pack()
            startMonthLabel.pack()
            startMonth.pack()
            startDateLabel.pack()
            startDate.pack()
            endYearLabel.pack()
            endYear.pack()
            endMonthLabel.pack()
            endMonth.pack()
            endDateLabel.pack()
            endDate.pack()
            retrieveData.pack()
            
    #creating a label and two radio button for th gui to retrieve the IMDB data
    dataOption = Label(root, text="Do you want to view yearly imdb data or monthly imdb data : ")
    yearlyList   = Radiobutton(root, text="View yearly data",padx = 20,variable=var,value ="1",command = selected)
    monthlyList  = Radiobutton(root, text="View monthly data",padx = 20,variable=var,value ="2",command = selected)


    dataOption.pack()
    yearlyList.pack()
    monthlyList.pack()

    root.mainloop()
    

#main login where the username and password and access code are been used to enter into the system

#loading the data stored in users.dat file
with open('users.dat','r') as file:
    userAccess = (file.read())
user = ast.literal_eval(userAccess)
tries = 0   
def login():
    global tries 
    for k,v in sorted(user.items(),key = lambda t:t[1]):
        if usernameGuess.get() in v:
            pin =user[k][1]
            accessCode =k
            if passwordGuess.get() == pin:
                messagebox.showinfo("Success","Login Successful !!\n\nYour access Code is "+str(accessCode))
               # messagebox.showinfo("Access Code","Your access Code-"+str(accessCode))
                accessText.pack()
                accessGuess.pack()
                accessData.pack()
            else:
                tries = tries + 1
                messagebox.showerror("Incorrect Pin","Please try again!!")
                if tries > 2:
                        messagebox.showerror("Exiting..","You have exceeded the number of limits !")
                        root.withdraw()
                        break
        elif (usernameGuess.get() == '' or passwordGuess.get() == ''):
            messagebox.showerror("Invalid","Please enter valid details !")
            break

def accessIMDB():
    for k,v in sorted(user.items(),key = lambda t:t[1]):
        if usernameGuess.get() in v:
            accessCode =str(k)
            if (accessGuess.get() == accessCode) :
                root.withdraw()
                root.destroy()
                viewIMDBData()
            break
        elif (accessGuess.get() =='' or usernameGuess.get() == '' or passwordGuess.get() == ''):
            messagebox.showerror("Invalid","Please enter complete details to get the access to the data")
            break 


#Gui Things
root = Tk()
root.title("Log-In")
root.geometry("800x750")

#Creating the username & password entry boxes
usernameText = Label(root, text="Username:")
usernameGuess = Entry(root)
passwordText = Label(root, text="Password:")
passwordGuess = Entry(root, show="*")
accessText = Label(root, text="Please enter access code:")
accessGuess = Entry(root)


#attempt to login button
attemptLogin = Button(text="Login", command= login)
accessData = Button(text="Access Data", command=accessIMDB)

usernameText.pack()
usernameGuess.pack()
passwordText.pack()
passwordGuess.pack()
attemptLogin.pack()
#Main Starter
root.mainloop()



Request:54; Frequency: 0.07984645755921907 requests/s


In [4]:
### SUMMARY STATS OF MEDIAN,HIGHEST & LOWEST VALUE OF VOTES,METASCORE AND IMDB RATINGS for th eyear between 2000 to 2018

#getting the median of the IMDB data
#the csv file was created by importing the data retrieved from the imdb website during run time
imdbmovielistBYYear = pd.read_csv('imdbListYear.csv')
medianOfRatingByYear = str(imdbmovielistBYYear['rating'].median())
medianOfMetascoreByYear = str(imdbmovielistBYYear['metascore'].median())
medianOfVotesByYear = str(imdbmovielistBYYear['votes'].median())
print ("Median of the IMDB Ratings for the years between 2000 and 2017 : " + medianOfRatingByYear+ "\nMedian of the Metascore for the years between 2000 and 2017 : " + medianOfMetascoreByYear
       +"\nMedian of the Votes for the years between 2000 and 2017 : " + medianOfVotesByYear)

#getting the maximum value of the IMDB data
maxOfRatingByYear = str(imdbmovielistBYYear['rating'].max())
maxOfMetascoreByYear = str(imdbmovielistBYYear['metascore'].max())
maxOfVotesByYear = str(imdbmovielistBYYear['votes'].max())
print ("\nMaximum Value of the IMDB Ratings for the years between 2000 and 2017 : " + maxOfRatingByYear+ "\nMaximum Value of the Metascore for the years between 2000 and 2017 : " + maxOfMetascoreByYear
       +"\nMaximum Value of the Votes for the years between 2000 and 2017 : " + maxOfVotesByYear)

#getting the minimum value of the IMDB data
minOfRatingByYear = str(imdbmovielistBYYear['rating'].min())
minOfMetascoreByYear = str(imdbmovielistBYYear['metascore'].min())
minOfVotesByYear =str(imdbmovielistBYYear['votes'].min())
print ("\nMinimum Value of the IMDB Ratings for the years between 2000 and 2017 : " + minOfRatingByYear+ "\nMinimum Value of the Metascore for the years between 2000 and 2017 : " + minOfMetascoreByYear
       +"\nMinimum Value of the Votes for the years between 2000 and 2017 : " + minOfVotesByYear)


Median of the IMDB Ratings for the years between 2000 and 2017 : 7.2
Median of the Metascore for the years between 2000 and 2017 : 64.0
Median of the Votes for the years between 2000 and 2017 : 273421.0

Maximum Value of the IMDB Ratings for the years between 2000 and 2017 : 9.0
Maximum Value of the Metascore for the years between 2000 and 2017 : 98
Maximum Value of the Votes for the years between 2000 and 2017 : 1947628

Minimum Value of the IMDB Ratings for the years between 2000 and 2017 : 1.9
Minimum Value of the Metascore for the years between 2000 and 2017 : 9
Minimum Value of the Votes for the years between 2000 and 2017 : 1745


In [4]:
### SUMMARY STATS OF MEDIAN,HIGHEST & LOWEST VALUE OF VOTES,METASCORE AND IMDB RATINGS for the date range between 2016/1/1 
##  to 2018/1/1

#getting the median of the IMDB data
#the csv file was created by importing the data retrieved from the imdb website during run time
imdbmovielistBYMonth = pd.read_csv('imdbListMonth.csv')
medianOfRatingByMonth = str(imdbmovielistBYMonth['rating'].median())
medianOfMetascoreByMonth = str(imdbmovielistBYMonth['metascore'].median())
medianOfVotesByMonth = str(imdbmovielistBYMonth['votes'].median())
print ("Median of the IMDB Ratings for the dates between 2016/1/1 and 2018/1/1 : " + medianOfRatingByMonth+ "\nMedian of the Metascore for the dates between 2016/1/1 and 2018/1/1 : " + medianOfMetascoreByMonth
       +"\nMedian of the Votes for the dates between 2016/1/1 and 2018/1/1 : " + medianOfVotesByMonth)

#getting the maximum value of the IMDB data
maxOfRatingByMonth = str(imdbmovielistBYMonth['rating'].max())
maxOfMetascoreByMonth = str(imdbmovielistBYMonth['metascore'].max())
maxOfVotesByMonth = str(imdbmovielistBYMonth['votes'].max())
print ("\nMaximum Value of the IMDB Ratings for the dates between 2016/1/1 and 2018/1/1 : " + maxOfRatingByMonth+ "\nMaximum Value of the Metascore for the dates between 2016/1/1 and 2018/1/1 : " + maxOfMetascoreByMonth
       +"\nMaximum Value of the Votes for the years dates 2016/1/1 and 2018/1/1 : " + maxOfVotesByMonth)

#getting the minimum value of the IMDB data
minOfRatingByMonth = str(imdbmovielistBYMonth['rating'].min())
minOfMetascoreByMonth = str(imdbmovielistBYMonth['metascore'].min())
minOfVotesByMonth =str(imdbmovielistBYMonth['votes'].min())
print ("\nMinimum Value of the IMDB Ratings for the dates between 2016/1/1 and 2018/1/1 : " + minOfRatingByMonth+ "\nMinimum Value of the Metascore for the dates between 2016/1/1 and 2018/1/1: " + minOfMetascoreByMonth
       +"\nMinimum Value of the Votes for the dates between 2016/1/1 and 2018/1/1 : " + minOfVotesByMonth)


Median of the IMDB Ratings for the dates between 2016/1/1 and 2018/1/1 : 6.3
Median of the Metascore for the dates between 2016/1/1 and 2018/1/1 : 58.0
Median of the Votes for the dates between 2016/1/1 and 2018/1/1 : 5055.5

Maximum Value of the IMDB Ratings for the dates between 2016/1/1 and 2018/1/1 : 8.0
Maximum Value of the Metascore for the dates between 2016/1/1 and 2018/1/1 : 94
Maximum Value of the Votes for the years dates 2016/1/1 and 2018/1/1 : 486181

Minimum Value of the IMDB Ratings for the dates between 2016/1/1 and 2018/1/1 : 3.6
Minimum Value of the Metascore for the dates between 2016/1/1 and 2018/1/1: 21
Minimum Value of the Votes for the dates between 2016/1/1 and 2018/1/1 : 10


In [5]:
############### EXTRA CREDITS ######################
################  1 .BOX PLOT ########################
#Box plot for Year vs Votes
imdbmovielistBYYear = pd.read_csv('imdbListYear.csv')
ax = sns.boxplot(x="year", y="votes", data=imdbmovielistBYYear)
sns.boxplot(x="year", y="votes", data=imdbmovielistBYYear).set_title("Year vs Votes")


Text(0.5,1,'Year vs Votes')

In [6]:
#Box plot for Year vs Ratings
ax = sns.boxplot(x="year", y="rating", data=imdbmovielistBYYear)
sns.boxplot(x="year", y="rating", data=imdbmovielistBYYear).set_title("Year vs Ratings")


Text(0.5,1,'Year vs Ratings')

In [7]:
#Box plot for Year vs Metascores
ax = sns.boxplot(x="year", y="metascore", data=imdbmovielistBYYear)
sns.boxplot(x="year", y="metascore", data=imdbmovielistBYYear).set_title("Year vs Metascore")

Text(0.5,1,'Year vs Metascore')

In [5]:
############### EXTRA CREDITS ######################
################  2 .DATABASE - SQLITE3  ########################

#the csv file was created by importing the data retrieved from the imdb website during run time
imdblist= csv.reader(open("imdbListYear.csv"))
next(imdblist)
con = sqlite3.connect(":memory:")
try:

    con.execute("create table SADAASREE(id,movie TEXT,year YEAR,rating FLOAT,metascore INTEGER,votes INTEGER)")
    con.executemany("insert into SADAASREE(id,movie,year,rating,metascore,votes) values (?, ?,?,?,?,?)", imdblist)

    for votes in con.execute("SELECT movie,votes FROM SADAASREE WHERE votes = (SELECT MAX(votes) FROM SADAASREE)  "):
        s = str (votes)
        start ="("
        end = ","
        my_string = s[s.find(start)+len(start):s.rfind(end)]
        print ("Movie with the Maximum number of votes in the IMDB website for the years between 2000 and 2017  : "+my_string)
        break

    for rating in con.execute("SELECT movie,rating FROM SADAASREE WHERE rating = (SELECT MAX(rating) FROM SADAASREE)  "):
        s = str (rating)
        start ="("
        end = ","
        my_string = s[s.find(start)+len(start):s.rfind(end)]
        print ("Movie with the Maximum rating in the IMDB website for the years between 2000 and 2017  : "+my_string)
        break

    for metascore in con.execute("SELECT movie,metascore FROM SADAASREE WHERE metascore = (SELECT MAX(metascore) FROM SADAASREE)  "):
        s = str (metascore)
        start ="("
        end = ","
        my_string = s[s.find(start)+len(start):s.rfind(end)]
        print ("Movie with the Maximum metascore in the IMDB website for the years between 2000 and 2017 : "+my_string)
        break

    for votes in con.execute("SELECT movie,votes FROM SADAASREE WHERE votes = (SELECT MIN(votes) FROM SADAASREE)  "):
        s = str (votes)
        start ="("
        end = ","
        my_string = s[s.find(start)+len(start):s.rfind(end)]
        print ("\nMovie with the Minimum number of votes in the IMDB website for the years between 2000 and 2017 : "+my_string)
        break

    for rating in con.execute("SELECT movie,rating FROM SADAASREE WHERE rating = (SELECT MIN(rating) FROM SADAASREE)  "):
        s = str (rating)
        start ="("
        end = ","
        my_string = s[s.find(start)+len(start):s.rfind(end)]
        print ("Movie with the Minimum rating in the IMDB website for the years between 2000 and 2017 : "+my_string)
        break

    for metascore in con.execute("SELECT movie,metascore FROM SADAASREE WHERE metascore = (SELECT MIN(metascore) FROM SADAASREE)  "):
        s = str (metascore)
        start ="("
        end = ","
        my_string = s[s.find(start)+len(start):s.rfind(end)]
        print ("Movie with the Minimum metascore in the IMDB website for the years between 2000 and 2017 : "+my_string)
        break

    for votes in con.execute("SELECT MAX(votes) AS maximumVotes FROM SADAASREE "):
        s = str (votes)
        start ="("
        end = ","
        my_string = s[s.find(start)+len(start):s.rfind(end)]
        print ("\nMaximum number of votes in the IMDB website for the years between 2000 and 2017 : "+ my_string)

    for votes in con.execute("SELECT MAX(metascore) AS maximumVotes FROM SADAASREE "):
        s = str (votes)
        start ="("
        end = ","
        my_string = s[s.find(start)+len(start):s.rfind(end)]
        print ("Maximum number of metascore in the IMDB website for the years between 2000 and 2017 : "+my_string)

    for rating in con.execute("SELECT MAX(rating) AS maximumVotes FROM SADAASREE "):
        s = str (rating)
        start ="("
        end = ","
        my_string = s[s.find(start)+len(start):s.rfind(end)]
        print ("Maximum number of rating in the IMDB website for the years between 2000 and 2017 : "+my_string)


    for votes in con.execute("SELECT MIN(votes) AS maximumVotes FROM SADAASREE "):
        s = str (votes)
        start ="("
        end = ","
        my_string = s[s.find(start)+len(start):s.rfind(end)]
        print ("\nMinimum number of votes in the IMDB website for the years between 2000 and 2017 : "+ my_string)

    for votes in con.execute("SELECT MIN(metascore) AS maximumVotes FROM SADAASREE "):
        s = str (votes)
        start ="("
        end = ","
        my_string = s[s.find(start)+len(start):s.rfind(end)]
        print ("Minimum number of metascore in the IMDB website for the years between 2000 and 2017 : "+my_string)

    for rating in con.execute("SELECT MIN(rating) AS maximumVotes FROM SADAASREE "):
        s = str (rating)
        start ="("
        end = ","
        my_string = s[s.find(start)+len(start):s.rfind(end)]
        print ("Minimum number of rating in the IMDB website for the years between 2000 and 2017 : "+my_string)
        
except:
     print("Unable to read data!")




Movie with the Maximum number of votes in the IMDB website for the years between 2000 and 2017  : 'The Dark Knight'
Movie with the Maximum rating in the IMDB website for the years between 2000 and 2017  : 'The Dark Knight'
Movie with the Maximum metascore in the IMDB website for the years between 2000 and 2017 : "Pan's Labyrinth"

Movie with the Minimum number of votes in the IMDB website for the years between 2000 and 2017 : 'Hot Summer Nights'
Movie with the Minimum rating in the IMDB website for the years between 2000 and 2017 : 'Disaster Movie'
Movie with the Minimum metascore in the IMDB website for the years between 2000 and 2017 : 'Battlefield Earth'

Maximum number of votes in the IMDB website for the years between 2000 and 2017 : 1947628
Maximum number of metascore in the IMDB website for the years between 2000 and 2017 : 98
Maximum number of rating in the IMDB website for the years between 2000 and 2017 : 9.0

Minimum number of votes in the IMDB website for the years between 